# Web-Scraping Code: 
## Part of RA work for Prof. Martin Fiszbein at Boston University (2020)
## See http://politicalgraveyard.com/index.html 

## Name: Nikhil Kumar

### 1. What does the code do? 
To scrape the name of politicians along with their birht county, state and religious denomiation from the website above.

### 2. How can you use it? 
In the 'EXE_PATH', you need to add the path of your own computer and then you can click on 'Run'. A new browser will open in your Computer containing the URL. The webiste is poorly organized in its HTML construction and I have used the pattern of rendering of data for retreiving the data that is required for analysis. 


### Import required functions and packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import requests

### Create an excel sheet for politicians name by birth state and county

In [ ]:
# Using Selenium for web-scraping
#EXE_PATH = r"F:/Winter 2020/RA Martin/Scraping_Summer_2020/chromedriver.exe"
#browser = webdriver.Chrome(executable_path=EXE_PATH)

curr = "http://politicalgraveyard.com/geo/plindex.html"

# open the webpage with all the links to different states
#browser.get(curr)
#browser.implicitly_wait(1)

html = urlopen(curr)
soup = BeautifulSoup(html, 'lxml')
# find all the links in the webpage
link_one = soup.findAll('a')

# counter for dictionary entries
count = 0

# dictionary for storage
dic_pol_name_state_county={}

for state_link in link_one:
# loop over the links in the webpage
# the strategy is to keep opening the 'geo' links until 'The Political Grveyard' link appears 
# then the loop breaks

    if('The Political Graveyard' in state_link.text):
        break
    elif(('geo' in state_link.get('href')) == True):

        # find the state name from the link text
        state = state_link.get_text().replace('\n', ' ')
        #print("STATE", state)

        if state=='Wyoming':
        # links in wyoming do not work on the website
            continue

        # open the link for each state    
        #browser.get(state_link.get('href'))
        #browser.implicitly_wait(1)

        html = urlopen(state_link.get('href'))
        soup = BeautifulSoup(html, 'lxml')

        # find the links in each webpages for each state
        link_two = soup.findAll('a')

        # loop over all the links in that state's webpage
        # the strategy is to keep opening the 'born' links until 'The Political Grveyard' link appears 
        # when the loop breaks

        for born_link in link_two:
            if('The Political Graveyard' in born_link.text):
                break
            elif(('born' in born_link.get('href')) == True):
                
                # open the link for each county
                #browser.get(born_link.get('href'))
                #browser.implicitly_wait(1)
                
                html = requests.get(born_link.get('href'))
                soup = BeautifulSoup(html.content)

                # create an identifier for county
                a = ['County', 'county', 'Borough', 'borough', 'census area', 'city', 'City','municipality', 'Miscellaneous', 'Parish', 'Municipio']

                # Now we find the name of the county
                for x in (soup.find('p')):
                    # check if any of the above county-identifier words are present in the county name tag
                    if (any(identifier in x for identifier in a)):
                        county = str(x).replace('\n','')
                        print(county)

                html = urlopen(born_link.get('href'))
                soup = BeautifulSoup(html, 'lxml')
                link_three = soup.findAll('a')

                # now loop over all the 'bio' links in the county's webpage
                # and extract all politician's name for that county
                # till 'The political Graveyard' link appears
                # when the loop breaks

                for bio_link in link_three:
                    if('The Political Graveyard' in bio_link.text):
                        break
                    elif(('bio' in bio_link.get('href')) == True):
                        # get the text the link which contains politicians names
                        pol_name = bio_link.get_text().replace('\n', ' ')

                        # save the data as a dictionary entry
                        dic_pol_name_state_county[count]={'state':state, 'county':county, 'pol_name':pol_name}
                        

                        # increase the counter
                        count += 1

In [ ]:
# save the dictionary into a dataframe
df_pol_name_state_county=pd.DataFrame.from_dict(dic_pol_name_state_county).T

# save the data frame into n excel sheet
df_pol_name_state_county.to_excel('data_born_pol.xlsx',engine='xlsxwriter')